In [ ]:
import os
import csv
import requests
import pandas as pd
import numpy as np
from datetime import datetime
from dotenv import load_dotenv

# set up api keys from env
load_dotenv()
api_key = os.getenv("ECOS_API_KEY")

# set up API base URL for ECOS
BASE_URL = "https://ecos.bok.or.kr/api"

In [ ]:
# set up API request template
def build_request_url(stats_code, start_date, end_date, item_code=None, format="json"):
    url = f"{BASE_URL}/StatisticSearch/{api_key}/{format}/kr/1/10000/{stats_code}/M/{start_date}/{end_date}"
    return url

# set up date range
start_date = "202401"
end_date = datetime.now().strftime("%Y%m")

## 4. 데이터 소스 설정

In [ ]:
# 데이터 수집 설정 (ECOS_data.py 참고)

# 단일 파일로 저장
single_files = [
    ("base_rate.csv", "722Y001", "M", "0101000", None),  # 기준금리
    ("cpi.csv", "901Y009", "M", "0", None),  # 소비자물가지수
    ("esi.csv", "513Y001", "M", None, ["경제심리지수(원계열)"]),  # 경제심리지수
    ("ccsi.csv", "511Y002", "M", None, ["현재생활형편CSI"]),  # 소비자심리지수
    ("construction_bsi_actual.csv", "512Y007", "M", None, ["업황실적BSI 1)"]),  # 건설업BSI(실적)
    ("construction_bsi_forecast.csv", "512Y008", "M", None, ["업황전망BSI 1)"]),  # 건설업BSI(전망)
    ("housing_sale_price.csv", "901Y062", "M", None, ["총지수"]),  # 주택매매가격지수
    ("housing_lease_price.csv", "901Y063", "M", None, ["총지수"]),  # 주택전세가격지수
    ("leading_index.csv", "901Y067", "M", None, ["선행지수순환변동치"]),  # 선행지수
]

# M2 증가율 (전기대비 증가율 계산)
m2_files = [
    ("m2_growth.csv", "101Y003", "M", None, ["M2(평잔, 계절조정계열)"])
]

# 개별 파일로 분리 (한 통계코드에 여러 항목)
individual_files = [
    ("market_rate", "721Y001", "M", None, ["국고채(3년)", "국고채(10년)", "회사채(3년, AA-)", "회사채(3년, BBB-)"]),  # 시장금리
    ("exchange_usd", "731Y006", "M", None, ["원/달러(종가 15:30)"]),  # 환율
    ("ppi", "404Y014", "M", None, ["철강1차제품", "비금속광물"]),  # 생산자물가지수
    ("import_price", "401Y015", "M", None, ["철강1차제품", "비금속광물"]),  # 수입물가지수
]

# 통계코드별 설명
STAT_CODES_INFO = {
    "722Y001": "한국은행 기준금리",
    "721Y001": "시장금리 (국고채, 회사채)",
    "731Y006": "원/달러 환율",
    "901Y009": "소비자물가지수",
    "404Y014": "생산자물가지수",
    "401Y015": "수입물가지수",
    "101Y003": "M2 통화량",
    "513Y001": "경제심리지수(ESI)",
    "511Y002": "소비자심리지수(CCSI)",
    "512Y007": "건설업BSI(실적)",
    "512Y008": "건설업BSI(전망)",
    "901Y067": "선행지수",
    "901Y062": "주택매매가격지수",
    "901Y063": "주택전세가격지수",
}

print("=== 수집할 경제지표 ===")
print(f"단일 파일: {len(single_files)}개")
print(f"M2 증가율: {len(m2_files)}개")
print(f"개별 파일: {len(individual_files)}개")
print(f"총 통계코드: {len(STAT_CODES_INFO)}개")

for code, desc in STAT_CODES_INFO.items():
    print(f"  {code}: {desc}")